<a href="https://colab.research.google.com/github/ZainHKazmi/Jupyter-Hierachy-Tool/blob/main/Jupyter_Hierachy_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupyter Hierachy Notebook

Jupyter Hieracy Notebook

The tool provides a concise method of synthesizing new assets into existing hierachies.

Initially, we create an ODBC connection with Avantis to query the avantis data. We load data in to a datafram and drop duplicates. We extract the dataset and generate a difaultdict representation of the dataset. NOTE: for POC, we utilize an excel instead of the database.

generate_edges is a function that maps relations between parent and child classes.

We then conduct a depth first travesal of the tree to determine whether any loops exists and if so, where.

Once the tree is validated, we allow the upload of new hierachies into the existing tree.

The script outputs a graphviz file which can be added to any graphviz processor and can be visualized in an tree image format

In [ ]:
import io
import pandas as pd
from google.colab import files
from collections import defaultdict
!pip install treelib
from treelib import Node, Tree
import codecs
import graphviz
import pyodbc

conn = pyodbc.connect(DRIVER = '{ODBC Driver 18 for SQL Server}',
                      SERVER = 'TWSQLP-EWMS.tw.toronto.ca\EWMS',
                      DATABASE = 'Database',
                      UID = 'integration_ro',
                      PWD = 'read0nly')

cursor = conn.cursor()

uploaded = files.upload()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unixodbc-dev is already the newest version (2.3.9-5).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
1 not fully installed or removed.
After this operation, 0 B of additional disk space will be used.
Setting up msodbcsql17 (17.8.1.1-1) ...
dpkg: error processing package msodbcsql17 (--configure):
 installed msodbcsql17 package post-installation script subprocess returned error exit status 127
Errors were encountered while processing:
 msodbcsql17
E: Sub-process /usr/bin/dpkg returned an error code (1)


Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [ ]:
tree_dict = pd.read_csv(io.BytesIO(uploaded['AIW.csv']))

In [ ]:
tree_dict = tree_dict.drop_duplicates(subset='Entity', keep="last")
graph = defaultdict(list)
def addEdge(graph,u,v):
    graph[u].append(v)

# definition of function
def generate_edges(graph):
    edges = []

    # for each node in graph
    for node in graph:

        # for each neighbour node of a single node
        for neighbour in graph[node]:

            # if edge exists then append
            edges.append((node, neighbour))
    return edges

for i in range(len(tree_dict)):
  addEdge(graph, tree_dict.iloc[i][1], tree_dict.iloc[i][0])

loop_dict = {}

def DFSHelper(graph, v, visited, recStack):
  visited[v] = True
  recStack[v] = True

  for count, neighbour in enumerate(graph[v]):
    if visited[neighbour] == False:
      if DFSHelper(neighbour, visited, recStack) == True:
        loop_dict[count] = visited
        return True
    elif recStack[neighbour] == True:
      loop_dict[count] = recStack
      return True

  recStack[v] = False
  return False

def DFS(graph):
  visited = [False] * (len(tree_dict.index) + 1)
  recStack = [False] * (len(tree_dict.index) + 1)
  for node in range(len(tree_dict.index)):
      if visited[node] == False:
          if DFSHelper(graph, node, visited, recStack) == True:
              return True
  return False

print(DFS(graph))

if not (DFS(graph)):
  tree = Tree()
  tree.create_node("parent", "parent")
  for key, value in graph.items():
    if type(key) == int:
      break
    try:
      tree.create_node(key, key, parent="parent")
    except:
      continue
    for val in value:
      if val != key:
        tree.create_node(val, val, parent=key)
str(tree)
tree.to_graphviz(filename="export.txt")
#codecs.decode(tree.show(), "utf-8")

False


In [ ]:
def insert_tree(tree, subtree):

  merged_graph = defaultdict(list)
  for key in set(tree.keys()).union(subtree.keys()):
    merged_graph[key] = tree.get(key, []) + subtree.get(key, [])

  return merged_graph


In [ ]:
sub_tree_file = files.upload()
sub_tree = pd.read_csv(io.BytesIO(sub_tree_file['new_tree.csv']))

sub_tree = sub_tree.drop_duplicates(subset='Entity', keep="last")

graph2 = defaultdict(list)
for i in range(len(sub_tree)):
  addEdge(graph, sub_tree.iloc[i][1], sub_tree.iloc[i][0])


new_tree = insert_tree(graph, sub_tree)

generate_edges(new_tree)

Saving new_tree.csv to new_tree.csv


ValueError: operands could not be broadcast together with shapes (0,) (128,) 